In [1]:
import os
if "models" not in os.listdir("."):
    os.chdir("..")

In [2]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [3]:
filename = "models/phi-3-16.gguf"
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained(filename, device_map="tpu:0")

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
import jax
import jax.numpy as jnp
prompt = """<|user|>
What is 3940 * 3892?<|end|>
<|assistant|>
"""
tokens = tokenizer.encode(prompt)
def tokens_to_array(tokens):
    token_array = jnp.asarray(tokens)
    if len(token_array) >= llama.mesh.shape["dp"]:
        token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    token_array = pz.nx.wrap(token_array, "batch", "seq")
    return token_array

In [6]:
from micrlhf.llama import LlamaBlock
from functools import partial


layer_source = 10
def make_get_resids(llama, layer_target):
    get_resids = llama.select().at_instances_of(LlamaBlock).pick_nth_selected(layer_target
                                                                              ).apply(lambda x:
        pz.nn.Sequential([
            pz.de.TellIntermediate.from_config(tag=f"resid_pre"),
            x
        ])
    )
    get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
    return get_resids
jittify = lambda x: partial(jax.jit(lambda lr, *args, **kwargs: lr(*args, **kwargs)[1][0].value), x)
get_resids_initial = make_get_resids(llama, layer_source)
get_resids_initial = jittify(get_resids_initial)

In [7]:
layer_target = 20
taker = jittify(make_get_resids(llama, layer_target).select().at_instances_of(LlamaBlock).apply_with_selected_index(
    lambda i, x: x if i >= layer_source else pz.nn.Identity()
).select().at_instances_of(pz.nn.EmbeddingLookup).apply(lambda _: pz.nn.Identity())
                .select().at_instances_of(pz.nn.ConstantRescale).pick_nth_selected(0).apply(lambda _: pz.nn.Identity()))

In [8]:
import dataclasses
import jax.numpy as jnp
from tqdm.auto import trange
import optax


inputs = llama.inputs.from_basic_segments(tokens_to_array([tokens]))
resid_initial = get_resids_initial(inputs)
bs = 16
seed = 9
iterations = 1000
scale = 10
def get_loss(rep):
    rep = pz.nx.wrap(rep, "batch", "embedding")
    rep = pz.nx.nmap(lambda a: a / jnp.linalg.norm(a) * scale)(rep.untag("embedding")).tag("embedding")
    resid = pz.nx.nmap(lambda x, y: x.at[-1].add(y))(resid_initial[{"batch": 0}].untag("seq"), rep).tag("seq")
    resid_out = taker(dataclasses.replace(inputs, tokens=resid,
                                          attention_mask=inputs.attention_mask,
                                          positions=inputs.positions))[{"seq": -1}]
    diffs = pz.nx.nmap(lambda x: (x[:, None] - x[None, :]) ** 2)(resid_out.untag("batch")).tag("batch1", "batch2")
    diffs = diffs.untag("embedding").sum()  # ** 0.5
    loss = diffs.data_array.mean()
    return -loss, (resid_out,)
start_add = jax.random.normal(jax.random.key(seed),
                              (bs, resid_initial.named_shape["embedding"]), dtype=resid_initial.dtype)

optimizer = optax.chain(
    optax.clip_by_global_norm(1.0),
    optax.adam(1e-2),
    optax.zero_nans(),
    
)
lwg = jax.value_and_grad(get_loss, has_aux=True)
# lwg = jax.jit(lwg)

# @partial(jax.jit, donate_argnums=(0, 1))
def train_step(addition, opt_state):
    (loss, (resid_out,)), grad = lwg(addition)
    updates, opt_state = optimizer.update(grad, opt_state, addition)
    addition = optax.apply_updates(addition, updates)
    return loss, addition, opt_state, dict(resid_out=resid_out)


In [9]:
addition = start_add
opt_state = optimizer.init(addition)
for _ in (bar := trange(iterations)):
    loss, addition, opt_state, aux = train_step(addition, opt_state)
    bar.set_postfix(loss=loss)

  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [9]:
from micrlhf.utils.load_sae import get_sae
from tqdm.auto import trange
from more_itertools import chunked
import numpy as np
sae = get_sae(layer_source, 4)

feat_bs = 64
feat_outs = []
try:
    for index in trange(0, len(sae["W_dec"]) // 4, feat_bs):
        rep = sae["W_dec"][index:index+feat_bs]
        rep = pz.nx.wrap(rep, "batch", "embedding")
        rep = pz.nx.nmap(lambda a: a / jnp.linalg.norm(a) * scale)(rep.untag("embedding")).tag("embedding")
        resid = pz.nx.nmap(lambda x, y: x.at[-1].add(y))(resid_initial[{"batch": 0}].untag("seq"), rep).tag("seq")
        resid_out = taker(dataclasses.replace(inputs, tokens=resid,
                                            attention_mask=inputs.attention_mask,
                                            positions=inputs.positions))[{"seq": -1}]
        feat_outs.append(np.array(resid_out.unwrap("batch", "embedding")))
except KeyboardInterrupt:
    pass
all_outs = np.concatenate(feat_outs, axis=0)
all_outs.shape

--2024-06-21 13:48:49--  https://huggingface.co/nev/phi-3-4k-saex-test/resolve/main/l10-test-run-4-2.73E-06/sae_weights.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 108.156.211.125, 108.156.211.90, 108.156.211.51, ...
Connecting to huggingface.co (huggingface.co)|108.156.211.125|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/eb/d8/ebd889d6ac58573e8e8a7aa1176d4d357581a6da60135b94aca378fddf4e9e54/c8cf3f62062b3cdc9509167526de244fa3271fca14dd9dc97f653ef970374bd7?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sae_weights.safetensors%3B+filename%3D%22sae_weights.safetensors%22%3B&Expires=1719236929&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxOTIzNjkyOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViL2Q4L2ViZDg4OWQ2YWM1ODU3M2U4ZThhN2FhMTE3NmQ0ZDM1NzU4MWE2ZGE2MDEzNWI5NGFjYTM3OGZkZGY0ZTllNTQvYzhjZjNmNj

  0%|          | 0/192 [00:00<?, ?it/s]

/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/jax/_src/ops/scatter.py:96: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=bfloat16 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(
/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/jax/_src/ops/scatter.py:96: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=bfloat16 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(


(12288, 3072)

In [10]:
# i love scipy. it has everything
from scipy.spatial.distance import pdist, squareform
from tqdm.auto import trange
distances = squareform(pdist(all_outs.astype(np.float16)))
existing = []
for _ in trange(bs // 2):
    md = distances.ravel().argmax()
    row, col = np.unravel_index(md, distances.shape)
    distances[row] = 0
    distances[col] = 0
    distances[:, row] = 0
    distances[:, col] = 0
    distances = distances + distances[row, None] / 2 + distances[:, row] / 2 + distances[col, None] / 2 + distances[:, col] / 2
    existing.extend([row, col])
vectors = sae["W_dec"][jnp.asarray(existing)]

  0%|          | 0/8 [00:00<?, ?it/s]

In [11]:
from micrlhf.utils.activation_manipulation import add_vector
from micrlhf.sampling import sample
import numpy as np

layer = layer_source
repeat = 4
act_add = add_vector(llama,
                     jnp.tile(vectors / jnp.linalg.norm(vectors, axis=-1, keepdims=True) * scale, (repeat, 1)),
                     layer)

texts, cached = sample(act_add, tokenizer, prompt,
       batch_size=bs * repeat, do_sample=True, return_model=True)
np.array(texts).reshape(repeat, bs).T.tolist()

  0%|          | 0/46 [00:00<?, ?it/s]

[['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculatingvations like "2c," would be calculated ch:\n\\[ 2 \\times 2! = 4 \\]\n\n?\n\n\\beginrintation<|end|><|assistant|> "multiplication" has',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The multiplication of 8 numbers is not something we should do here, but I can guide you through the process in a more simplified manner.\nv math tool or a calculator could be used for such extensive multiplication.\nAs',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculating yacmulation _ 6 _ _ _ _ _ _ _ _ _ _, please use the multiplication operation. The result for 6980 * 1992 is as follows:\n ',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> 19942220.<|end|><|assistant|> Actually, your calculation is incorrect. The multiplication of 98,00v by 99,4d0 is 9772220'],
 ['<s><|user|> What is 3940 * 3892?<|end|><|assistant|>s:\n.\n\n The multiplication is:\n    3940\n x  3892\n-----------\n   713600\n+ 274320\n-----------',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|>}\nThe = :\n3940\n\\overline{3892} * \n=\n\n* \n\\overline{3892} =\n3940\n\\_8',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|>}\n3940 * 3892 = \n\nHere is the calculation:\n: \n1s place =  * 8 +  * 2 =  +  5\n: ',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|>s  of :\n\n3940 * 3892 = \n\n   3940\n  x 3892\n\n -- - ----\n  2048  '],
 ['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The multiplication of 3940 and 3892 is: 3940 * 3892 = 11429280.<|end|><|end|><|assistant|> Here is the solution using',
  "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The multiplication of 3940 and 3892 is 11,485,280.\n\nHere's the calculation:\n\n```\n  3940\nx",
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The multiplication of 3940 by 3892 is:\n\n3940 * 3892 = 11,371,680\n\nSo, the product',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The multiplication of 3940 and 3892 is:\n\n3940 * 3892 = 11431680\n\nSo, the result of '],
 ['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> 3940 * 3892 = 15434240\nTherefore, 3940 * 3892 = 15434240',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> 3940 * 3892 = 15,430,880.<|end|><|assistant|> [calculated using long multiplication]<|end|><|assistant|> To multiply 3940 by 38',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> 3940 * 3892 = 15421280<|end|><|assistant|> 3940 multiplied by 3892 equals 15421280',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> 3940 * 3892 = 15403680.\n<|end|><|assistant|> 15403680.\nTherefore, 3940 multiplied'],
 ['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> 656633280<|end|><|assistant|> """\nTo calculate the product of 504133280, you can perform the multiplication:\n\n56663318',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> 65007328<|end|><|assistant|> $1$ dollars, $$0$$ thousand dollars, $$, $$3$ millions, $$0$$ thousands, and $$0$$ cions.\n\nAlthough',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> 256412800.<|end|><|assistant|> here,\nThe result of multiplying 1 (1 thousand) by 3 (3 hundred thousand) is 100 thousand,\nthe result of',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To multiply these two numbers together, you would use the standard multiplication method, multiplying each digit of one number by each digit of the other number and carrying over the results accordingly.\n\nHowever, as we are performing this operation'],
 ['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 multiplied by 3892 is 15265280.<|end|><|endoftext|><|endoftext|><|endoftext|> message>\nFind the area of the region bounded by the graphs of the',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> 3940 * 3892 = 15201280.<|end|><|endoftext|>before the 15th) = 2 * 2 * $250 = $10',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 15258880.<|end|><|endoftext|>5" or "😂".\nI\

In [ ]:
from micrlhf.utils.activation_manipulation import add_vector
from micrlhf.sampling import sample
import numpy as np

layer = layer_source
repeat = 4
act_add = add_vector(llama,
                     jnp.tile(addition / jnp.linalg.norm(addition, axis=-1, keepdims=True) * scale, (repeat, 1)),
                     layer)

texts, cached = sample(act_add, tokenizer, prompt,
       batch_size=bs * repeat, do_sample=True, return_model=True)
np.array(texts).reshape(repeat, bs).T.tolist()

  0%|          | 0/46 [00:00<?, ?it/s]

[['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> We can use the standard algorithm for multiplication to find the product of 3940 and 3892.\nFirst, multiply 3940 by 2 (the ones digit of 389',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To multiply these two numbers, we can use the standard algorithm for multiplication.\n\n      3940\n    x 3892\n    -----\n    15648  (394',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To multiply these two numbers, we can use the standard algorithm for multiplication:\n\n      3940\n    x 3892\n  --------\n     157680 (394',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> We can use the standard algorithm for multiplication to solve this problem.\nFirst, we multiply the units digit of 3940 (which is 0) with the units digit of 3892 (which is'],
 ['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> ??? - Rounded\n===\n<|assistant|> css\n<|assistant|> }\n<|assistant|> }\n<|assistant|> css\n<|assistant|> }\n<|assistant|> }\n<|assistant|> math\n<|assistant|> }\n<|assistant|> math\n<|assistant|> }\n<|assistant|> }\n<|assistant|> {childNodes:',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> ?"\n\nSince you\'ve only provided numbers, I\'ll calculate the product for you:\n\n3940 * 3892 = 15,242,800\n',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> ?"\nsupport: "The calculation of the multiplication problem is suitable for self-generation with a potential numerical answer, assuming that the operations are correct and feasible. However, the answer should be verified through independent calculation,',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> ?"\n\nTo calculate the result, we simply multiply 3940 by 3892.\n\n3940 * 3892 = 15,214,8'],
 ['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> give me a moment to calculate it.\n\n[Calculator]\n\n3940 * 3892 = 15248480\n\nHere is the result of the multiplication:',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> both large numbers. Just to confirm, the multiplication is straightforward:\n\n3940 * 3892 = 15,331,200\n\nThis product should equal the multiplication of',
  "<s><|user|> What is 3940 * 3892?<|end|><|assistant|>mu math\nHere's how you can calculate it:\n\nMultiplying large numbers can be done easily with a calculator or using a multiplication algorithm. Since we're in a text-based format, I'",
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> both the number of calculations involved and the formatting of the response.\n\n3940 multiplied by 3892 equals 15,138,480. This is a straightforward arithmetic calculation'],
 ['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 15263680.<|end|><|assistant|> Final answer: 15263680\n\nAnswer: 15',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> \\textbf{Answer:}15452480\n<|end|><|assistant|> To find the product of 3940 and 3892, you multiply the two numbers together:\n\n3940',
  "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The answer is 14755680.\n\nHere's the calculation:\n\n3940 * 3892 = 14755680\n\nSo",
  "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> Correct! 3940 * 3892 equals 15150880.\n\nHere's the step-by-step calculation:\n3940\nx 3"],
 ['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate 3940 * 3892, you multiply the two numbers together.\n\n\n3940\n\nx 3892\n\n--------\n\n7880 (',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To multiply 3940 by 3892, you can use the standard algorithm for multiplication:\n\n\n1. Write down the two numbers with the larger one on top (3892) and',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To multiply 3940 by 3892, we can apply the standard algorithm for multiplication:\n\n\n```\n\n     3940\n\n×    3892\n\n----------\n',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To find the product of 3940 and 3892, you would simply multiply the two numbers together:\n\n\n3940 * 3892 = 15,34'],


In [ ]:
from micrlhf.utils.activation_manipulation import add_vector
from micrlhf.sampling import sample
import numpy as np

layer = layer_source
repeat = 4
act_add = add_vector(llama,
                     jnp.tile(addition / jnp.linalg.norm(addition, axis=-1, keepdims=True) * scale, (repeat, 1)),
                     layer)

texts, cached = sample(act_add, tokenizer, prompt,
       batch_size=bs * repeat, do_sample=True, return_model=True)
np.array(texts).reshape(repeat, bs).T.tolist()

In [ ]:
from micrlhf.utils.load_sae import get_sae
from micrlhf.utils.ito import grad_pursuit
resid_out = aux["resid_out"]
refusal = (resid_out[{"batch": -4}] - resid_out.untag("batch").mean()).unwrap("embedding")
sae = get_sae(20, 5)
k = 2
w, recon = grad_pursuit(refusal, sae["W_dec"], k, pos_only=True)
i = jax.lax.top_k(jnp.abs(w), k)[1]
((recon - refusal) ** 2).mean(), i, w[i]

--2024-05-30 04:37:42--  https://huggingface.co/nev/phi-3-4k-saex-test/resolve/main/l20-test-run-5-7.00E-06/sae_weights.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 108.156.211.90, 108.156.211.95, 108.156.211.51, ...
Connecting to huggingface.co (huggingface.co)|108.156.211.90|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/eb/d8/ebd889d6ac58573e8e8a7aa1176d4d357581a6da60135b94aca378fddf4e9e54/83ac196e9d76d5a3db26b4cc47737ff5c79d3cba0ce3954d02283b9331464948?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sae_weights.safetensors%3B+filename%3D%22sae_weights.safetensors%22%3B&Expires=1717303062&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNzMwMzA2Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViL2Q4L2ViZDg4OWQ2YWM1ODU3M2U4ZThhN2FhMTE3NmQ0ZDM1NzU4MWE2ZGE2MDEzNWI5NGFjYTM3OGZkZGY0ZTllNTQvODNhYzE5NmU5

(<jax.Array(23.02443, dtype=float32)>,
 <jax.Array([20279, 23629], dtype=int32)>,
 <jax.Array([44.070187, 39.419018], dtype=float32)>)

In [ ]:
from micrlhf.utils.activation_manipulation import set_direction


layer = layer_target
repeat = 64
vector = refusal
# vector = recon
# act_add = add_vector(llama,
#                     (vector / jnp.linalg.norm(vector, axis=-1, keepdims=True))[None, :] * jnp.linspace(-200, 200, repeat)[:, None],
#                      layer)
act_add = set_direction(llama, vector, jnp.linspace(-100, 100, repeat), layer)

sample(act_add, tokenizer, prompt,
       batch_size=repeat, do_sample=False)

  0%|          | 0/46 [00:00<?, ?it/s]

['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 1, poorer-than-anything-I-think-of-in-response-to-your-question, but I',
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 1,199,551. However, without the exact calculation, I can't provide the precise total, but it'",
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 11,199,968. To get the exact answer, the product of 3940 and 38',
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 1,199,551. However, without the exact calculation, I can't provide the precise answer, but it'",
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 11,999,888. To get the exact answer, the product of 3940 and 38',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 11,999,888. To calculate it exactly, you can multiply the two numbers:\n\n3940',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 11,999,888. To get the exact answer, the calculation is:\n\n3940 * ',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 11,993,328. To calculate it exactly, you can multiply the two numbers:\n\n3940',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 11,993,328. To calculate it exactly, you can multiply the two numbers:\n\n3940',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 11,059,368. To calculate it exactly, you can multiply the two numbers:\n\n3940',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 11 breeding ground for mosquitoes. However, the actual product of those numbers is not related to mosquitoes. The product',
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 11,139,360. Without knowing the specific numbers you're asking about, I can't provide a more detailed",
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 1,173,068. Without a calculator, it's hard to calculate that quickly, but if you're interested",
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 1,1 breeding 9,408, which is not possible as the product of two numbers is always a specific value. However,',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 1,1 breeding 9,408,720.\n\nTo calculate it, you can multiply the two numbers:\n',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 1,1 breeding 9,408,720.\n\nTo calculate it, you can multiply the two numbers:\n',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 1,149,328. To calculate this, you can multiply the two numbers:\n\n3940 * 3',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 1,146,236. To calculate this, you can multiply the two numbers:\n\n3940 * 3',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 1,143,328. To calculate this, you can multiply the two numbers:\n\n3940 * 3',
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 15364320.<|end|><|assistant|> Yes, that's correct! The product of 3940 and 38",
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 15364320.<|end|><|assistant|> Yes, that's correct. The calculation is as follows:\n\n3940",
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 15363360.<|end|><|assistant|> Yes, that's correct. The calculation is as follows:\n\n3940",
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> T

In [ ]:
from micrlhf.utils.load_sae import get_sae
from micrlhf.utils.ito import grad_pursuit
refusal_steer = addition[-4]
sae_steer = get_sae(10, 4)
k = 16
w, recon_steer = grad_pursuit(refusal_steer, sae_steer["W_dec"], k, pos_only=True)
i = jax.lax.top_k(jnp.abs(w), k)[1]
((recon_steer - refusal_steer) ** 2).mean(), i, w[i]

--2024-05-30 04:39:06--  https://huggingface.co/nev/phi-3-4k-saex-test/resolve/main/l10-test-run-4-2.73E-06/sae_weights.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 108.156.211.95, 108.156.211.51, 108.156.211.90, ...
Connecting to huggingface.co (huggingface.co)|108.156.211.95|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/eb/d8/ebd889d6ac58573e8e8a7aa1176d4d357581a6da60135b94aca378fddf4e9e54/c8cf3f62062b3cdc9509167526de244fa3271fca14dd9dc97f653ef970374bd7?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sae_weights.safetensors%3B+filename%3D%22sae_weights.safetensors%22%3B&Expires=1717303146&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNzMwMzE0Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViL2Q4L2ViZDg4OWQ2YWM1ODU3M2U4ZThhN2FhMTE3NmQ0ZDM1NzU4MWE2ZGE2MDEzNWI5NGFjYTM3OGZkZGY0ZTllNTQvYzhjZjNmNjIw

(<jax.Array(1.0404294, dtype=float32)>,
 # jax.Array int32(16,) [≥694, ≤46_558] nonzero:16
   Array([38392, 21968,  7395, 35911, 46558, 17895,  8265, 30491, 37354,
          19255, 13932, 11776,  9762,   694, 41052, 14909], dtype=int32)
 ,
 # jax.Array float32(16,) ≈3.8 ±0.41 [≥3.4, ≤5.0] nonzero:16
   Array([4.993171 , 4.415919 , 4.1850085, 3.9498377, 3.9282415, 3.8214664,
          3.8211484, 3.7118008, 3.6665282, 3.6618662, 3.5518913, 3.4981477,
          3.4750347, 3.469136 , 3.4441085, 3.4338064], dtype=float32)
 )

In [ ]:
layer = layer_source
repeat = 64
# vector = addition[-4]
vector = recon_steer
act_add = add_vector(llama,
                     (vector / jnp.linalg.norm(vector, axis=-1, keepdims=True))[None, :] * jnp.linspace(-scale * 2, scale * 2, repeat)[:, None],
                     layer)

sample(act_add, tokenizer, prompt,
       batch_size=repeat, do_sample=False)

  0%|          | 0/46 [00:00<?, ?it/s]

['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\nFirst, multiply the ones',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\nFirst, multiply the ones',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\nFirst, multiply 3',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\nFirst, multiply 3',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\n15760',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\n15760',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\n15760',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\n15760',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\n15760',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\n15760',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\n15720',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n_____\n1576',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n_____\n1576',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n_____\n1576',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n_____\n1572',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n_____\n1576',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n_____\n1572',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n_____\n1572',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 15,361,280. You can calculate this by multiplying the two numbers together:\n\n3940\n',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 15,361,280. You can calculate this by multiplying the two numbers together:\n\n3940\n',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 15,365,280. You can calculate this by multiplying the two numbers together:\n\n3940\n',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 15,361,280. You can calculate this by multiplying the two numbers together:\n\n3940\n',
 '<s><|user|> What is 3940 * 3892?<|end|>

: 